In [19]:
import requests
import time
from datetime import datetime
import json
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [20]:
url = "https://www.99acres.com/search/property/rent/indira-nagar-bangalore-east?city=22&locality=375&preference=R&area_unit=1&res_com=R"

In [21]:
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36"

In [22]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
chrome_options.add_argument(f'user_agent={user_agent}')
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)
driver.get(url)
wait = WebDriverWait(driver, 10)

In [23]:
element = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.ID, "2")))

In [24]:
element.click()

In [25]:
response = driver.page_source
soup = BeautifulSoup(response, "lxml")

script_tags = soup.find_all("script", {"type": "application/ld+json"})

property_links = []
for script_tag in script_tags:
    json_data = json.loads(script_tag.string)
    if "@type" in json_data and json_data["@type"] == "Apartment" and "url" in json_data:
        property_links.append(json_data["url"])
for link in property_links:
    print(link)


https://www.99acres.com/2-bhk-bedroom-independent-builder-floor-for-rent-in-stage-1-indira-nagar-bangalore-east-1100-sq-ft-spid-W68898648
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-stage-2-indira-nagar-bangalore-east-1250-sq-ft-spid-O68910562
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1230-sq-ft-spid-L67457250
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1200-sq-ft-spid-T67894920
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1000-sq-ft-r2-spid-B66200990
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1950-sq-ft-spid-W68292914
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1300-sq-ft-r30-spid-R30584477
https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-indira-nagar-bangalore-east-1440-sq-ft-r26-spid-E35001367
https

In [26]:
next_page_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//a[@class='list_header_bold']")))
next_page_url = next_page_link.get_attribute('href')
driver.get(next_page_url)
time.sleep(5)

response = driver.page_source
soup = BeautifulSoup(response, "lxml")

script_tags = soup.find_all("script", {"type": "application/ld+json"})
property_links_next_page = []

for script_tag in script_tags:
    json_data = json.loads(script_tag.string)
    if "@type" in json_data and json_data["@type"] == "Apartment" and "url" in json_data:
        property_links_next_page.append(json_data["url"])


all_property_links = property_links + property_links_next_page

In [27]:
all_property_links

['https://www.99acres.com/2-bhk-bedroom-independent-builder-floor-for-rent-in-stage-1-indira-nagar-bangalore-east-1100-sq-ft-spid-W68898648',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-stage-2-indira-nagar-bangalore-east-1250-sq-ft-spid-O68910562',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1230-sq-ft-spid-L67457250',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1200-sq-ft-spid-T67894920',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1000-sq-ft-r2-spid-B66200990',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1950-sq-ft-spid-W68292914',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-1300-sq-ft-r30-spid-R30584477',
 'https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-indira-nagar-bangalore-east-1440-

In [28]:
data = []
for link in all_property_links:
    try:
        driver.get(link)
        time.sleep(5)

        # Extract the rent
        rent_element = driver.find_element(By.ID, 'pdPrice')
        rent = float(rent_element.text.replace(',', ''))

        # Extract the listed date
        listed_date_element = driver.find_element(By.ID, 'postedOnAndByLabel')
        listed_date_str = listed_date_element.text.split('on ')[-1]
        listed_date = datetime.strptime(listed_date_str, '%b %d, %Y').strftime('%d-%m-%Y')

        # The property link is already known

        # Extract the carpet area
        carpet_area_element = driver.find_element(By.ID, 'carpetArea_span')
        carpet_area = float(carpet_area_element.text.split(' ')[0])

        # Calculate the price per square foot
        price_per_sqft = rent / carpet_area

        data.append([rent, listed_date, link, carpet_area, price_per_sqft])

    except Exception as e:
        print(f"Skipping property {link} due to error: {e}")


Skipping property https://www.99acres.com/2-bhk-bedroom-apartment-flat-for-rent-in-stage-2-indira-nagar-bangalore-east-1250-sq-ft-spid-O68910562 due to error: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="carpetArea_span"]"}
  (Session info: headless chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x005C8893+48451]
	(No symbol) [0x0055B8A1]
	(No symbol) [0x00465058]
	(No symbol) [0x00490467]
	(No symbol) [0x0049069B]
	(No symbol) [0x004BDD92]
	(No symbol) [0x004AA304]
	(No symbol) [0x004BC482]
	(No symbol) [0x004AA0B6]
	(No symbol) [0x00487E08]
	(No symbol) [0x00488F2D]
	GetHandleVerifier [0x00828E3A+2540266]
	GetHandleVerifier [0x00868959+2801161]
	GetHandleVerifier [0x0086295C+2776588]
	GetHandleVerifier [0x00652280+612144]
	(No symbol) [0x00564F6C]
	(No symbol) [0x005611D8]
	(No symbol) [0x005612BB]
	(No symbol) [0x00554857]
	BaseThreadInitThunk [0x75C37D59+25]
	RtlInitializeExceptionChain [0x76F4B74B+107]
	RtlClearBit

In [29]:
df = pd.DataFrame(data, columns=['Rent', 'Listed Date', 'Property Link', 'Carpet Area', 'Price per sqft'])

In [30]:
df

,Rent,Listed Date,Property Link,Carpet Area,Price per sqft
0,30000.0,04-05-2023,https://www.99acres.com/2-bhk-bedroom-independ...,1100.0,27.272727
1,52000.0,22-04-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1100.0,47.272727
2,50000.0,28-04-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1100.0,45.454545
3,36000.0,04-05-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1000.0,36.000000
4,60000.0,01-04-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1750.0,34.285714
5,42000.0,08-05-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1300.0,32.307692
6,44000.0,08-05-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1400.0,31.428571
7,38000.0,09-05-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1300.0,29.230769
8,33000.0,29-04-2023,https://www.99acres.com/2-bhk-bedroom-apartmen...,1100.0,30.000000
9,32000.0,28-04-2023,https://www.99acres.com/2-bhk-bedroom-independ...,1200.0,26.666667


In [31]:
df.to_excel('99acres_data.xlsx', index=False)

In [32]:
oldest_listed_property = df['Listed Date'].min()
property_with_max_rent = df.loc[df['Rent'].idxmax()]['Property Link']
property_with_min_rent = df.loc[df['Rent'].idxmin()]['Property Link']

In [33]:
print(f'Oldest Listed Property: {oldest_listed_property}')
print(f'Property with Maximum Rent: {property_with_max_rent}')
print(f'Property with Minimum Rent: {property_with_min_rent}')

Oldest Listed Property: 01-04-2023
Property with Maximum Rent: https://www.99acres.com/3-bhk-bedroom-apartment-flat-for-rent-in-hal-2nd-stage-bangalore-east-2200-sq-ft-spid-O68285258
Property with Minimum Rent: https://www.99acres.com/2-bhk-bedroom-independent-builder-floor-for-rent-in-indira-nagar-bangalore-east-563-sq-ft-r1-spid-E66573992
